In [ ]:
! pip install langchain openai tiktoken chromadb einops transformers xformers sentencepiece accelerate bitsandbytes docx2txt python-docx sentence_transformers
! pip install chromadb==0.3.27
! pip install pydantic==1.10.11
#! pip install --upgrade protobuf==3.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, warnings, torch
from transformers import (
    LlamaTokenizer,
    LlamaForCausalLM,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    pipeline
)
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import Docx2txtLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.cache import InMemoryCache
from chromadb.config import Settings

llm_cache = InMemoryCache()

warnings.filterwarnings("ignore")

os.environ["LLM_REPOSITORY"] = "lmsys/vicuna-13b-v1.3"#"lmsys/vicuna-13b-delta-v0"
os.environ["EMBEDDING_REPOSITORY"] = "sentence-transformers/all-MiniLM-L6-v2"
os.environ["MAX_TOKENS"] = "1024"
os.environ["VECTOR_STORE_FOLDER"] = "vectordb"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(os.environ["LLM_REPOSITORY"])

model = AutoModelForCausalLM.from_pretrained(
    os.environ["LLM_REPOSITORY"],
    load_in_8bit=True,
    device_map="auto",
)
# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=int(os.environ["MAX_TOKENS"]),
    temperature=0.6,
    top_p=0.15,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipeline)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# model_kwargs = dict(max_length=int(os.environ["MAX_TOKENS"]), temperature=0.6, load_in_8bit=True)

# local_llm = HuggingFacePipeline.from_model_id(
#     model_id=os.environ["LLM_REPOSITORY"],
#     task="text-generation",
#     model_kwargs=model_kwargs,
#   )

In [ ]:
doc_path = "/content/Preliminary_Sample_Summary_For_Few_Shot.docx"
loader = Docx2txtLoader(file_path=doc_path)
doc = loader.load()

BadZipFile: ignored

In [ ]:
prompt_template = """:

"{text}"


SUMMARY:"""
summary_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])


summary_templates = [
    "Provide a summary of the clinical trial for a patient.",
    "You are the PI of the trial. Summarize the trial for a potential patient at a 6th-grade reading level.",
    "You are an oncologist. Describe this trial for your patient at a 6th-grade reading level.",
    "with Chain-of-Thought: Provide a summary of the clinical trial for a patient. Please explain step-by-step."
]

summary_prompts = [PromptTemplate.from_template(f"{template} {prompt_template}") for template in summary_templates]

In [ ]:
for i, summary_prompt in enumerate(summary_prompts):
  summary_chain = load_summarize_chain(local_llm, chain_type="map_reduce", combine_prompt=summary_prompt)
  print(f"PROMPT: {summary_templates[i]}\n")
  print(f"SUMARY: {summary_chain.run(doc)}\n\n")

PROMPT: Provide a summary of the clinical trial for a patient.



SUMARY:  The INSIGHT study is comparing two treatment options for patients with HR+ metastatic breast cancer that falls outside of the Luminal A subtype. This study will evaluate the effectiveness of each treatment option as well as examine how specific genetic changes in the tumors may affect response to therapy and the development of drug resistance.


PROMPT: You are the PI of the trial. Summarize the trial for a potential patient at a 6th-grade reading level.

SUMARY:  The INSIGHT study wants to find out if one treatment or another works better for people with advanced breast cancer that doesn't fit into the Luminal A category. They will also look at how genetic changes in their tumors might affect how well these treatments work.


PROMPT: You are an oncologist. Describe this trial for your patient at a 6th-grade reading level.

SUMARY:  The INSIGHT study is comparing two treatments for advanced breast cancer that have certain characteristics. It wants to see which treatment works 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_kwargs = {'device': device}
encode_kwargs = {"normalize_embeddings": False}
embedding = HuggingFaceEmbeddings(
    model_name=os.environ["EMBEDDING_REPOSITORY"],
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
if (
    not os.path.isdir(os.environ["VECTOR_STORE_FOLDER"])
    or
    len(os.listdir(os.environ["VECTOR_STORE_FOLDER"])) == 0):
    docsearch_db = Chroma.from_documents(
        documents=doc,
        persist_directory=os.environ["VECTOR_STORE_FOLDER"],
        embedding=embedding)

    # persist embeddings to ROM
    docsearch_db.persist()
    docsearch_db = None

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory=os.environ["VECTOR_STORE_FOLDER"],
    anonymized_telemetry=False,
)

docsearch_db = Chroma(
    embedding_function=embedding,
    client_settings=CHROMA_SETTINGS)

retriever = docsearch_db.as_retriever(search_kwargs={"k":1})

In [ ]:
prompt_str = "Answer this question: {question}"
qa_prompt = PromptTemplate.from_template(prompt_str)
qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [ ]:
queries = [
  "What is the purpose of this study?",
  "What conditions must patients have who are eligible for this trial?",
  "What are the age or gender/sex requirements for this clinical trial?",
  "What drugs will trial subjects be subjected to?",
  "Is this trial still available for enrollment? (basically is current date between start date and end date)",
]

for i, query in enumerate(queries):
    query = qa_prompt.format_prompt(question=query)
    print(f"answer to question {i+1}")
    print(qa(query.text)["result"])
    print("\n")

answer to question 1
 The primary objective of this study is to determine the impact of early chemotherapy (capecitabine) versus endocrine therapy-based regimens on anti-tumor effects in patients with non-Luminal A hormone receptor-positive (HR+) metastatic breast cancer. Secondary objectives include comparing the safety and tolerability of these treatments and determining the impact of early chemotherapy on anti-tumor effects. Correlative studies will examine whether tumor mutations detected in circulating free DNA (cfDNA) are early surrogates of response and whether cfDNA results at disease progression reveal new genomic alterations possibly related to resistance to therapy.


answer to question 2
 Patients must be diagnosed with non-Luminal A hormone receptor-positive (HR+) metastatic breast cancer to be eligible for this trial.


answer to question 3
 The text does not provide specific information about age or gender/sex requirements for this clinical trial.


answer to question 4
